In [1]:
!nvidia-smi

Mon Dec 21 16:33:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz   


In [4]:
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz

# install findspark 
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [6]:
!wget https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_train.parquet
!wget https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_test.parquet

--2020-12-21 16:34:21--  https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_train.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6827427 (6.5M) [application/octet-stream]
Saving to: ‘covtype_train.parquet’

covtype_train.parqu 100%[===================>]   6.51M  --.-KB/s    in 0.05s   

2020-12-21 16:34:22 (127 MB/s) - ‘covtype_train.parquet’ saved [6827427/6827427]

--2020-12-21 16:34:22--  https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_test.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response.

In [7]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
!wget http://insecure.repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/0.1.0/rapids-4-spark_2.12-0.1.0.jar

--2020-12-21 16:34:22--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145993287 (139M) [application/java-archive]
Saving to: ‘cudf-0.14-cuda10-1.jar’

cudf-0.14-cuda10-1. 100%[===================>] 139.23M   279MB/s    in 0.5s    

2020-12-21 16:34:23 (279 MB/s) - ‘cudf-0.14-cuda10-1.jar’ saved [145993287/145993287]

--2020-12-21 16:34:23--  https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231556205 (221M) [application/java-archive]
Saving to: ‘xgboost4j_3.0-1.0.0-0.1.0.jar’

xgboo

In [8]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.14-cuda10-1.jar,/content/xgboost4j_3.0-1.0.0-0.1.0.jar,/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar,/content/rapids-4-spark_2.12-0.1.0.jar pyspark-shell'

In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.plugins", "com.nvidia.spark.SQLPlugin").config("spark.rapids.memory.gpu.pooling.enabled",False).getOrCreate()


In [10]:
spark.sparkContext.addPyFile('/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar')
spark.sparkContext.addPyFile('/content/rapids-4-spark_2.12-0.1.0.jar')

In [11]:
import pyarrow.parquet as pq
pq.read_table('covtype_train.parquet')
pq_file=pq.read_table('covtype_train.parquet')

In [12]:
reader = spark.read

In [13]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark import XGBoostRegressionModel, XGBoostRegressor

import numpy as np
import pandas as pd

In [14]:
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
label="target"
features = [ x for x in pq_file.column_names if x != label ]

In [15]:
train_data = reader.format('parquet').load('covtype_train.parquet')
test_data = reader.format('parquet').load('covtype_test.parquet')

In [16]:
train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wilder

In [17]:

import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 2
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [18]:
%%time
model=classifier.fit(train_data)

CPU times: user 49.7 ms, sys: 11.3 ms, total: 61 ms
Wall time: 16.5 s


In [19]:
result=model.transform(test_data)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Make predictions.
predictions = model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
accuracy

0.72992719574534

In [20]:
spark.stop()

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [22]:
x_df_train = pd.read_parquet('covtype_train.parquet', engine='pyarrow')
X_train = x_df_train.loc[:, x_df_train.columns != 'target']
y_train = x_df_train["target"]
x_df_test = pd.read_parquet('covtype_test.parquet', engine='pyarrow')
X_test = x_df_test.loc[:, x_df_test.columns != 'target']
y_test = x_df_test["target"]


In [23]:
%%time

model = XGBClassifier( eta= 0.1,
    gamma= 0.1,
    missing= 0.0,
    treeMethod= 'gpu_hist',
    maxDepth= 6, 
    growPolicy= 'depthwise',
    lambda_= 1.0,
    subsample= 1.0,
    numRound= 1000,
    numWorkers= 1,
    verbosity= 2)
model.fit(X_train, y_train)

[16:35:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:35:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:35:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nod

In [24]:
sk_pred = model.predict(X_test)
sk_pred = np.round(sk_pred)
sk_acc = round(accuracy_score(y_test, sk_pred), 2)
print("XGB accuracy using Sklearn:", sk_acc*100, '%')

XGB accuracy using Sklearn: 74.0 %


In [25]:
%%time

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 2,
    'n_gpus': 1
}

params.update(params)

clf = xgb.train(params, dtrain)

[16:39:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[16:39:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[16:39:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[16:39:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[16:39:46] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[16:39:48] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 2 pruned nodes, max_depth=6
[16:39:49] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[16:39:50] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 e

In [26]:
rapids_pred = clf.predict(dtest)
rapids_pred = np.round(rapids_pred)
rapids_acc = round(accuracy_score(y_test, rapids_pred), 2)
print("XGB accuracy using RAPIDS:", rapids_acc*100, '%')

XGB accuracy using RAPIDS: 64.0 %
